In [8]:
%config IPCompleter.greedy=True

In [26]:
import pandas as pd
import numpy as np

In [27]:
merge = pd.read_csv('merge.csv')
quick = pd.read_csv('quick.csv')

In [28]:
print(merge.columns)

Index(['input_size', 'comparisons_num'], dtype='object')
